In [1]:
# Import the dependencies.
import pandas as pd 
import gmaps
import requests
import json
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Georgetown,5.4112,100.3354,78.89,92,20,3.44,MY,2022-10-02 22:50:12
1,1,Albany,42.6001,-73.9662,52.88,65,100,8.50,US,2022-10-02 22:50:41
2,2,Longyearbyen,78.2186,15.6401,37.24,93,75,3.44,SJ,2022-10-02 22:55:07
3,3,Puerto Ayora,-0.7393,-90.3518,73.36,91,85,4.00,EC,2022-10-02 22:55:07
4,4,East London,-33.0153,27.9116,66.85,93,23,10.38,ZA,2022-10-02 22:55:08


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)

In [6]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds,
dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.

fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind,
dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.

fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Georgetown,5.4112,100.3354,78.89,92,20,3.44,MY,2022-10-02 22:50:12
10,10,Kapaa,22.0752,-159.3190,82.18,81,85,7.00,US,2022-10-02 22:55:09
16,16,Butaritari,3.0707,172.7902,82.35,75,66,16.51,KI,2022-10-02 22:55:12
21,21,Isangel,-19.5500,169.2667,78.91,89,100,3.44,VU,2022-10-02 22:55:14
29,29,Bubaque,11.2833,-15.8333,80.73,78,83,8.03,GW,2022-10-02 22:55:18
31,31,Avarua,-21.2078,-159.7750,77.05,61,62,21.85,CK,2022-10-02 22:55:18
32,32,Samana,19.2056,-69.3369,78.76,84,100,7.70,DO,2022-10-02 22:55:19
33,33,Victoria,22.2855,114.1577,83.32,88,14,5.01,HK,2022-10-02 22:55:19
36,36,Hilo,19.7297,-155.0900,88.29,77,0,4.00,US,2022-10-02 22:55:20
42,42,Biak,-0.9131,122.8766,77.68,82,90,1.39,ID,2022-10-02 22:55:23


In [12]:
preferred_cities_df.count()

City_ID       175
City          175
Lat           175
Lng           175
Max Temp      175
Humidity      175
Cloudiness    175
Wind Speed    175
Country       175
Date          175
dtype: int64

In [13]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Georgetown,MY,78.89,5.4112,100.3354,
10,Kapaa,US,82.18,22.0752,-159.3190,
16,Butaritari,KI,82.35,3.0707,172.7902,
21,Isangel,VU,78.91,-19.5500,169.2667,
29,Bubaque,GW,80.73,11.2833,-15.8333,
31,Avarua,CK,77.05,-21.2078,-159.7750,
32,Samana,DO,78.76,19.2056,-69.3369,
33,Victoria,HK,83.32,22.2855,114.1577,
36,Hilo,US,88.29,19.7297,-155.0900,
42,Biak,ID,77.68,-0.9131,122.8766,


In [16]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

{'html_attributions': [],
 'next_page_token': 'AcYSjRgfxr2gNmsEeohacdEihlI4aYRdsjKDR3HOeEPLZNo2YvdZ-KqPVm4-WmH8XBDsNld7vHN-1DlRaCIZt89-Bks64u_PD37SbM7NzZkAVKSv4sv4HA4chF4J3kPcIyTpusRGRvuPy4D_zIPUFW8AJqYANzpzyh2eOUaW2N3EmAzfF7sxRdkVk1wn40D6csklVo6cCAkH8Xpx7RRMgXP9b8MkISiVMwy7XSvx6HaCUoNfhFKShmBb3c6MfEPaYkzqBwWGmtm3Ey3PxvIk-j_K5ACaOApN5HeYlrn4lC5LsU-l92t0HWinevpSV1QxvyQGA3z1sQtN6ci7-9xwV4baDCzDsGCioPI5PWG3G-da42Yzq864nrh63tMbziPKJ0avOTpzq2-90b-9yRJbTsjgCDv42aI-5V__5XikZq4v8mp3VLV-e1TXq7_B',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594360302915,
      'lng': 2.354355480291502},
     'southwest': {'lat': 48.8567380697085, 'lng': 2.351657519708498}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'nam

In [17]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError, NewError):
    print("Hotel not found... skipping.")
hotels

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 31.454014, 'lng': 131.217192},
    'viewport': {'northeast': {'lat': 31.4553163302915,
      'lng': 131.2184753302915},
     'southwest': {'lat': 31.4526183697085, 'lng': 131.2157773697085}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'name': 'Hanaoka',
   'photos': [{'height': 2448,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/103395140733297477063">nobi maru</a>'],
     'photo_reference': 'AcYSjRips8b5xdclQYU5q2pS-igei5M425zBj2tiyejHKXljP1g619IB5dKvgr0CSAFgfn5m2p3t7FPFq1qyaaPVhy4-EITpNcJosOSpvM8e27tv7H-PLFquHVN_iNCpwpWDjPY_K4BW2zdHRFDgy5A-2aK63HXFymZmQoNlz0G3aFJG5dRP',
     'width': 3264}],
   'place_id': 'ChIJkXwm2cvNPjURtKVKnrC-_Ek',
   'plus_code': {'compound_code'

In [19]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Georgetown,MY,78.89,5.4112,100.3354,
10,Kapaa,US,82.18,22.0752,-159.3190,
16,Butaritari,KI,82.35,3.0707,172.7902,
21,Isangel,VU,78.91,-19.5500,169.2667,
29,Bubaque,GW,80.73,11.2833,-15.8333,
31,Avarua,CK,77.05,-21.2078,-159.7750,
32,Samana,DO,78.76,19.2056,-69.3369,
33,Victoria,HK,83.32,22.2855,114.1577,
36,Hilo,US,88.29,19.7297,-155.0900,
42,Biak,ID,77.68,-0.9131,122.8766,


In [20]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [27]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [28]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))